In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_excel('data_new.xlsx')
df

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,Date,DateClaimed,Date_Diff
0,12,5,3,Honda,Urban,2,1,1,Female,Single,...,No,External,none,1 year,3 to 4,1994,Liability,1994-12-29,1994-01-05,-358
1,1,3,3,Honda,Urban,1,1,4,Male,Single,...,No,External,none,no change,1 vehicle,1994,Collision,1994-01-20,1994-01-25,5
2,10,5,5,Honda,Urban,4,11,2,Male,Married,...,No,External,none,no change,1 vehicle,1994,Collision,1994-10-29,1994-11-11,13
3,6,2,6,Toyota,Rural,5,7,1,Male,Married,...,No,External,more than 5,no change,1 vehicle,1994,Liability,1994-06-12,1994-07-02,20
4,1,5,1,Honda,Urban,2,2,2,Female,Single,...,No,External,none,no change,1 vehicle,1994,Collision,1994-02-01,1994-02-09,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15414,11,4,5,Toyota,Urban,2,11,5,Male,Married,...,No,External,none,no change,1 vehicle,1996,Collision,1996-11-23,1996-12-04,11
15415,11,5,4,Pontiac,Urban,5,12,1,Male,Married,...,No,External,more than 5,no change,3 to 4,1996,Liability,1996-11-29,1996-12-07,8
15416,11,5,4,Toyota,Rural,5,12,1,Male,Single,...,No,External,1 to 2,no change,1 vehicle,1996,Collision,1996-11-29,1996-12-07,8
15417,12,1,1,Toyota,Urban,4,12,2,Female,Married,...,No,External,more than 5,no change,1 vehicle,1996,All Perils,1996-12-03,1996-12-13,10


In [4]:
X = df[['AccidentArea',
        'Sex',
        'MaritalStatus',
        'Age', 
        'Fault', 
        'VehicleCategory', 
        'VehiclePrice', 
        'RepNumber', 
        'Deductible', 
        'DriverRating', 
        'Days_Policy_Accident',
        'Days_Policy_Claim',
        'PastNumberOfClaims',
        'AgeOfVehicle',
        'PoliceReportFiled',
        'WitnessPresent',
        'AgentType',
        'NumberOfSuppliments',
        'AddressChange_Claim',
        'NumberOfCars',
        'BasePolicy',
        'Date_Diff']]

    

y = df['FraudFound_P']

print(X.shape)
print(y.shape)

(15419, 22)
(15419,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


categorical_columns =  ['AccidentArea',
                        'Sex',
                        'MaritalStatus',
                        'Fault', 
                        'VehicleCategory', 
                        'VehiclePrice',
                        'Days_Policy_Accident',
                        'Days_Policy_Claim',
                        'PastNumberOfClaims',
                        'AgeOfVehicle',
                        'PoliceReportFiled',
                        'WitnessPresent',
                        'AgentType',
                        'NumberOfSuppliments',
                        'AddressChange_Claim',
                        'NumberOfCars',
                        'BasePolicy']

numerical_columns = ['Age', 'RepNumber','Deductible', 'DriverRating', 'Date_Diff']


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns),
        ('num', StandardScaler(), numerical_columns)
    ],
    remainder='passthrough' 
)

# Step 4: Fit and transform X_train
X_train_encoded = preprocessor.fit_transform(X_train)

# Step 5: Transform X_test using the same transformer
X_test_encoded = preprocessor.transform(X_test)

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


model = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', max_iter = 999999999, random_state=42)
model.fit(X_train_encoded, y_train)
y_pred = model.predict(X_train_encoded)
y_test_pred = model.predict(X_test_encoded)
accuracy = accuracy_score(y_train, y_pred)
print("Train Accuracy:", accuracy)

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300],
    'tol': [1e-4, 1e-3, 1e-2]
}

grid_search = GridSearchCV(model, param_grid, cv=10, scoring='recall')
grid_search.fit(X_train_encoded, y_train)

# Step 5: Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Step 6: Get the best model
best_model = grid_search.best_estimator_

# Step 7: Make predictions on the test set using the best model
y_pred2 = best_model.predict(X_train_encoded)
y_test_pred2 = best_model.predict(X_test_encoded)

# Step 8: Calculate accuracy on the test set
accuracy_test = accuracy_score(y_test, y_test_pred2)
print("Test Accuracy:", accuracy_test)


Train Accuracy: 0.6239325478326667


d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anacond

Best Hyperparameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}
Test Accuracy: 0.6027885862516212


d:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1080 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "d:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solver newton-cg supports only 'l2' or '

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
accuracy_train = accuracy_score(y_train, y_pred)
precision_train = precision_score(y_train, y_pred)
recall_train = recall_score(y_train, y_pred)
roc_auc_train = roc_auc_score(y_train, model.predict_proba(X_train_encoded)[:, 1])
confusion_matrix_train = confusion_matrix(y_train, y_pred)

accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, model.predict_proba(X_test_encoded)[:, 1])
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

print("Training Set Metrics (Not Tuned):")
print("Accuracy:", accuracy_train)
print("Precision:", precision_train)
print("Recall:", recall_train)
print("ROC AUC:", roc_auc_train)
print("Confusion Matrix:\n", confusion_matrix_train)

print("\nTest Set Metrics (Not Tuned):")
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("ROC AUC:", roc_auc_test)
print("Confusion Matrix:\n", confusion_matrix_test)


accuracy_train_combined = accuracy_score(y_train, y_pred2)
precision_train_combined = precision_score(y_train, y_pred2)
recall_train_combined = recall_score(y_train, y_pred2)
roc_auc_train_combined = roc_auc_score(y_train, best_model.predict_proba(X_train_encoded)[:, 1])
confusion_matrix_train_combined = confusion_matrix(y_train, y_pred2)

accuracy_test_combined = accuracy_score(y_test, y_test_pred2)
precision_test_combined = precision_score(y_test, y_test_pred2)
recall_test_combined = recall_score(y_test, y_test_pred2)
roc_auc_test_combined = roc_auc_score(y_test, best_model.predict_proba(X_test_encoded)[:, 1])
confusion_matrix_test_combined = confusion_matrix(y_test, y_test_pred2)

print("Training Set Metrics (Tuned):")
print("Accuracy:", accuracy_train_combined)
print("Precision:", precision_train_combined)
print("Recall:", recall_train_combined)
print("ROC AUC:", roc_auc_train_combined)
print("Confusion Matrix:\n", confusion_matrix_train_combined)

print("\nTest Set Metrics (Tuned):")
print("Accuracy:", accuracy_test_combined)
print("Precision:", precision_test_combined)
print("Recall:", recall_test_combined)
print("ROC AUC:", roc_auc_test_combined)
print("Confusion Matrix:\n", confusion_matrix_test_combined)

Training Set Metrics (Not Tuned):
Accuracy: 0.9407631607393795
Precision: 0.2857142857142857
Recall: 0.007380073800738007
ROC AUC: 0.8156297150294962
Confusion Matrix:
 [[8699   10]
 [ 538    4]]

Test Set Metrics:
Accuracy: 0.9361219195849546
Precision: 0.16666666666666666
Recall: 0.0051813471502590676
ROC AUC: 0.7860521217356705
Confusion Matrix:
 [[2886    5]
 [ 192    1]]
Training Set Metrics (Not Tuned):
Accuracy: 0.9414117392714301
Precision: 0.0
Recall: 0.0
ROC AUC: 0.4958395458911531
Confusion Matrix:
 [[8709    0]
 [ 542    0]]

Test Set Metrics:
Accuracy: 0.9374189364461738
Precision: 0.0
Recall: 0.0
ROC AUC: 0.46964942119817976
Confusion Matrix:
 [[2891    0]
 [ 193    0]]


d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
